In [ ]:
# %% [markdown]
# # DGCNN: Direct 3D Point Cloud Classification
# - FPS sampling
# - Augmentation
# - Training, Evaluation, Metrics, and Visualization

# %% Imports & Setup
import os, math, time, random
from types import SimpleNamespace
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import sys
sys.path.append('../Utils')  # adjust if your models/configs are elsewhere
import configs
from models import DGCNN

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
print("Device:", device)

# Directories
TRAIN_SET = configs.TRAIN_DIR
TEST_SET = configs.TEST_DIR
MODELS_SAVE_DIR = configs.MODEL_DIR

# Config
CFG = SimpleNamespace(
    num_points=1024,
    k=20,
    emb_dims=1024,
    dropout=0.5,
    batch_size=16,
    epochs=80,
    patience=12,
    base_lr=1e-3,
    min_lr=1e-5,
    weight_decay=1e-4,
    label_smoothing=0.1,
    num_workers=4,
    seed=42
)

# Seed
def set_seed(s=42):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed(CFG.seed)


Device: cpu


In [ ]:

# # FPS Sampling, Normalization, and Augmentations

# %%
def farthest_point_sampling(points: np.ndarray, n_samples: int) -> np.ndarray:
    N = points.shape[0]
    if N == 0:
        return np.zeros((n_samples, 3), dtype=np.float32)
    n_samples = min(n_samples, N)
    centroids = np.zeros((n_samples,), dtype=np.int64)
    distances = np.ones((N,), dtype=np.float32) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(n_samples):
        centroids[i] = farthest
        centroid = points[farthest]
        dist = np.sum((points - centroid) ** 2, axis=1)
        sel = dist < distances
        distances[sel] = dist[sel]
        farthest = np.argmax(distances)
    sampled = points[centroids]
    if sampled.shape[0] < CFG.num_points:
        pad_idx = np.random.choice(len(sampled), CFG.num_points - len(sampled), replace=True)
        sampled = np.concatenate([sampled, sampled[pad_idx]], axis=0)
    return sampled.astype(np.float32)

def normalize_unit(points: np.ndarray) -> np.ndarray:
    c = points.mean(axis=0, keepdims=True)
    p = points - c
    scale = np.max(np.sqrt((p**2).sum(1))) + 1e-8
    return (p / scale).astype(np.float32)

def random_jitter(points: np.ndarray, sigma=0.01, clip=0.05) -> np.ndarray:
    noise = np.clip(sigma * np.random.randn(*points.shape), -clip, clip).astype(np.float32)
    return (points + noise).astype(np.float32)

def random_scale(points: np.ndarray, low=0.85, high=1.2) -> np.ndarray:
    s = np.random.uniform(low, high)
    return (points * s).astype(np.float32)

def random_rotation_z(points: np.ndarray) -> np.ndarray:
    theta = np.random.uniform(0, 2*np.pi)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]], dtype=np.float32)
    return (points @ R.T).astype(np.float32)

def load_txt_points(path: Path) -> np.ndarray:
    pts = np.loadtxt(path).astype(np.float32)
    if pts.ndim == 1: pts = pts[None, :]
    if pts.shape[1] > 3: pts = pts[:, :3]
    return pts

def list_point_files(root: Path):
    files = []
    for ext in ("*.pts", "*.xyz", "*.txt"):
        files.extend(sorted((root).rglob(ext)))
    return files


In [ ]:
# %% [markdown]
# # Dataset & DataLoader

# %%
class PointCloudDataset(Dataset):
    def __init__(self, root_dir: Path, num_points=1024, augment=False):
        self.root_dir = Path(root_dir)
        self.num_points = num_points
        self.augment = augment
        self.classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {c:i for i,c in enumerate(self.classes)}
        self.samples = []
        for cls in self.classes:
            for f in list_point_files(self.root_dir/cls):
                self.samples.append((f, self.class_to_idx[cls]))
        if len(self.samples) == 0:
            raise RuntimeError(f"No point files found under {root_dir}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        pts = load_txt_points(path)
        pts = farthest_point_sampling(pts, self.num_points)
        pts = normalize_unit(pts)
        if self.augment:
            pts = random_rotation_z(pts)
            pts = random_scale(pts)
            pts = random_jitter(pts)
        return torch.from_numpy(pts.T), torch.tensor(label, dtype=torch.long)

# Create Datasets and Loaders
train_ds = PointCloudDataset(TRAIN_SET, num_points=CFG.num_points, augment=True)
test_ds  = PointCloudDataset(TEST_SET,  num_points=CFG.num_points, augment=False)

num_classes = len(train_ds.classes)
print(f"Classes ({num_classes}): {train_ds.classes}")

train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True,
                          num_workers=CFG.num_workers, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=CFG.batch_size, shuffle=False,
                          num_workers=CFG.num_workers, pin_memory=True)


In [ ]:
# %% [markdown]
# # Model, Optimizer, Scheduler, Loss

# %%
# Model
model = DGCNN(num_classes=num_classes, k=CFG.k, emb_dims=CFG.emb_dims, dropout=CFG.dropout)
model = model.to(device)

# Loss with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=CFG.label_smoothing)

# Optimizer (AdamW for stability)
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.base_lr, weight_decay=CFG.weight_decay)

# Cosine Annealing LR Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=CFG.epochs, eta_min=CFG.min_lr
)

print(f"Model Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"Optimizer: AdamW | Scheduler: CosineAnnealingLR")


In [ ]:
# %% [markdown]
# # Training and Evaluation Functions

# %%
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0
    for points, labels in loader:
        points, labels = points.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        total_loss += loss.item() * labels.size(0)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)
    return total_loss / total_samples, total_correct / total_samples


@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0
    all_labels, all_preds = [], []
    for points, labels in loader:
        points, labels = points.to(device), labels.to(device)
        outputs = model(points)
        loss = criterion(outputs, labels)
        preds = outputs.argmax(dim=1)

        total_loss += loss.item() * labels.size(0)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    return (
        total_loss / total_samples,
        total_correct / total_samples,
        np.array(all_labels),
        np.array(all_preds)
    )


In [ ]:
# %% [markdown]
# # Training Loop with Early Stopping and History Tracking

# %%
train_losses, val_losses, train_accs, val_accs = [], [], [], []
best_val_acc = 0.0
patience_counter = 0
best_model_wts = None

for epoch in range(CFG.epochs):
    start_time = time.time()

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc, val_labels, val_preds = evaluate(model, test_loader, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    scheduler.step()

    elapsed = time.time() - start_time
    print(f"[Epoch {epoch+1:02d}/{CFG.epochs}] "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} | "
          f"Time: {elapsed:.1f}s")

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = model.state_dict()
        patience_counter = 0
        torch.save(best_model_wts, MODELS_SAVE_DIR / "best_dgcnn.pth")
    else:
        patience_counter += 1
        if patience_counter >= CFG.patience:
            print("Early stopping triggered.")
            break

# Load best model
if best_model_wts is not None:
    model.load_state_dict(best_model_wts)
print(f"Best Val Accuracy: {best_val_acc:.4f}")


In [ ]:
# %% [markdown]
# # Final Evaluation and Classification Metrics

# %%
val_loss, val_acc, val_labels, val_preds = evaluate(model, test_loader, criterion)
print(f"Final Test Loss: {val_loss:.4f} | Final Test Accuracy: {val_acc:.4f}")

print("\nClassification Report:")
print(classification_report(val_labels, val_preds, target_names=train_ds.classes))

cm = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(6,6))
plt.imshow(cm, cmap="Blues")
plt.title("Confusion Matrix")
plt.colorbar()
plt.xticks(np.arange(num_classes), train_ds.classes, rotation=45)
plt.yticks(np.arange(num_classes), train_ds.classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
# %% [markdown]
# # Training History Visualization

# %%
epochs_ran = len(train_losses)
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(range(1, epochs_ran+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs_ran+1), val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(1, epochs_ran+1), train_accs, label='Train Acc')
plt.plot(range(1, epochs_ran+1), val_accs, label='Val Acc')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training & Validation Accuracy")
plt.legend()

plt.show()
